This notebook analyses the total generation (in MWh), total generation revenue (in AUD) and the volume-weighted average price for various fuel types in the National Electricity Market (NEM).

The input data includes:
1. Spot prices for all regions of the NEM, with 5-minute resolution, covering the period from FY2020 to FY2024.
    -- ##Production\Report\Electricity\Generation\Actual\Region\Metered\Generation Stack FuelType ScheduleType (5min)
2. Dispatched electricity by fuel type across all NEM regions, with 5-minute resolution, for the same FY2020 to FY2024 period.
    -- ##Production\Report\Electricity\Price\Actual\NEM\Dispatch (5min)\Dispatch Price all regions (5min)

These analyses aim to provide insights into generation performance and pricing dynamics by fuel type in the NEM.

In [2]:
import pandas as pd
from os import path
import numpy as np
import glob

In [8]:
input_path = r'C:\Users\wwang2\Documents\GitHub\NEM-generation\data'
clear_output = r'C:\Users\wwang2\Documents\GitHub\NEM-generation\results'

In [9]:
l=glob.glob(path.join(input_path, '*_2019_2024_5_minutes.csv'))   

In [17]:
df = pd.read_csv(path.join(input_path, "Spot Price_2019_2024.csv"))
print(df.head())

        DateTime  QLD1 DISPATCH_PRICE  NSW1 DISPATCH_PRICE  \
0  1/7/2019 0:05                64.80                67.26   
1  1/7/2019 0:10                56.67                58.07   
2  1/7/2019 0:15                57.45                58.71   
3  1/7/2019 0:20                56.24                57.68   
4  1/7/2019 0:25                56.49                57.92   

   VIC1 DISPATCH_PRICE  SA1 DISPATCH_PRICE  TAS1 DISPATCH_PRICE  
0                64.46               65.90                88.49  
1                54.25               54.99                51.78  
2                54.91               55.90                52.16  
3                52.89               53.91                49.76  
4                49.10               51.73                47.84  


In [ ]:
def ToD(df):
    df['DateTime']=pd.to_datetime(df['DateTime'],format='%d/%m/%Y %H:%M')
    df['Month']=df['DateTime'].dt.month
    df['CY']=df['DateTime'].dt.year
    df['Quarter']=df['DateTime'].dt.quarter
    df['FY']=df['DateTime'].map(lambda d: d.year + 1 if d.month > 6 else d.year)
    df['Hour']=df['DateTime'].dt.hour

In [19]:
ToD(df)
print(df.head())

             DateTime  QLD1 DISPATCH_PRICE  NSW1 DISPATCH_PRICE  \
0 2019-07-01 00:05:00                64.80                67.26   
1 2019-07-01 00:10:00                56.67                58.07   
2 2019-07-01 00:15:00                57.45                58.71   
3 2019-07-01 00:20:00                56.24                57.68   
4 2019-07-01 00:25:00                56.49                57.92   

   VIC1 DISPATCH_PRICE  SA1 DISPATCH_PRICE  TAS1 DISPATCH_PRICE  Month    CY  \
0                64.46               65.90                88.49      7  2019   
1                54.25               54.99                51.78      7  2019   
2                54.91               55.90                52.16      7  2019   
3                52.89               53.91                49.76      7  2019   
4                49.10               51.73                47.84      7  2019   

   Quarter    FY  Hour  
0        3  2020     0  
1        3  2020     0  
2        3  2020     0  
3        3  2020

In [ ]:
def VWAP(df_price,df_generation, state):
    
    df_generation=df_generation.fillna(0)   # Formating the generation data,1, convert na into 0, 2, generation data devide by 12, 5 minutes data
    df_generation[df_generation.filter(regex='DISPATCH_MW').keys()]=df_generation[df_generation.filter(regex='DISPATCH_MW').keys()]*1/12
   
    df_generation.columns = df_generation.columns.str.replace(r' DISPATCH_MW', '')                         # 3, Rename the data
    
    
    for e in Fuel:                                                   # Check whether it contains all the fuel type
        if df_generation.columns.tolist().count(e)==0:               # if it missed one fuel type, then generation set up as 0
            df_generation['{}'.format(e)]=0
        else:
            pass
    
    df=df_price[['DateTime', '{} DISPATCH_PRICE'.format(state)]].merge(df_generation[Fuel], how='inner', on='DateTime')   # merge generation and price data together
    
    df['{}_Solar_Revenue'.format(state)]=df['{} DISPATCH_PRICE'.format(state)]*df['Solar']
    df['{}_Wind_Revenue'.format(state)]=df['{} DISPATCH_PRICE'.format(state)]*df['Wind']    
        
    ToD(df)# change the datetime type and get FY, CY, HOUR and Qtly
    
    df_FY=df.groupby('FY').sum().reset_index()                                             
    
    df_FY['{}_Solar_VWAP'.format(state)]=df_FY['{}_Solar_Revenue'.format(state)]/df_FY['Solar']
    df_FY['{}_Wind_VWAP'.format(state)]=df_FY['{}_Wind_Revenue'.format(state)]/df_FY['Wind']
    #df_FY['{}_Total_VWAP'.format(state)]=df_FY['{}_Total_Revenue'.format(state)]/df_FY['Gen_Total']
    
    df_FY=df_FY.iloc[:,[0]+list(range(-2,0))] # Remove unnecessary information
    
    df_FY_Hour=df.groupby(['FY','Hour']).sum().reset_index()
    
    df_FY_Hour['{}_Solar_VWAP'.format(state)]=df_FY_Hour['{}_Solar_Revenue'.format(state)]/df_FY_Hour['Solar']
    df_FY_Hour['{}_Wind_VWAP'.format(state)]=df_FY_Hour['{}_Wind_Revenue'.format(state)]/df_FY_Hour['Wind']
    
    
    df_FY_Hour=df_FY_Hour.iloc[:,list(range(0,2))+list(range(-2,0))]   # Remove unnecessary information
    
    
    return (df_FY, df_FY_Hour)
    